# C4.5

http://gabrielelanaro.github.io/blog/2016/03/03/decision-trees.html

In [24]:
import pandas as pd
import numpy as np

# eps for making value a bit greater than 0 later on
eps = np.finfo(float).eps

from numpy import log2 as log


Creating a dataset,

In [5]:
# The input values
x1 = [0, 1, 1, 2, 2, 2,3,2,1]
x2 = [0, 0, 1, 1, 1, 0,2,2,1]
# The class
y = np.array([0, 0, 0, 1, 1, 0,1,0,1])

In [6]:
def partition(a):
    return {c: (a==c).nonzero()[0] for c in np.unique(a)}

In [7]:
def entropy(s):
    res = 0
    val, counts = np.unique(s, return_counts=True)
    freqs = counts.astype('float')/len(s)
    for p in freqs:
        if p != 0.0:
            res -= p * np.log2(p)
    return res

In [8]:
def mutual_information(y, x):

    res = entropy(y)

    # We partition x, according to attribute values x_i
    val, counts = np.unique(x, return_counts=True)
    freqs = counts.astype('float')/len(x)

    # We calculate a weighted average of the entropy
    for p, v in zip(freqs, val):
        res -= p * entropy(y[x == v])

    return res

In [9]:
def mutual_information(y, x):
    res = entropy(y)

    # We partition x, according to attribute values x_i
    val, counts = np.unique(x, return_counts=True)
    freqs = counts.astype('float')/len(x)

    # We calculate a weighted average of the entropy
    for p, v in zip(freqs, val):
        res -= p * entropy(y[x == v])

    return res

In [10]:
def is_pure(s):
    return len(set(s)) == 1

def recursive_split(x, y):
    # If there could be no split, just return the original set
    if is_pure(y) or len(y) == 0:
        return y

    # We get attribute that gives the highest mutual information
    gain = np.array([mutual_information(y, x_attr) for x_attr in x.T])
    selected_attr = np.argmax(gain)

    # If there's no gain at all, nothing has to be done, just return the original set
    if np.all(gain < 1e-6):
        return y


    # We split using the selected attribute
    sets = partition(x[:, selected_attr])

    res = {}
    for k, v in sets.items():
        y_subset = y.take(v, axis=0)
        x_subset = x.take(v, axis=0)

        res["x_%d = %d" % (selected_attr, k)] = recursive_split(x_subset, y_subset)

    return res

X = np.array([x1, x2]).T

In [11]:
c4_5 = recursive_split(X, y)

In [12]:
def pretty(d, indent=0):
    for key, value in d.items():
        print('\t' * indent + str(key))

        if isinstance(value, dict):
            pretty(value, indent+1)
        else:
            print('\t' * (indent+1) + str(value))

In [13]:
pretty(c4_5)

x_1 = 0
	[0 0 0]
x_1 = 1
	x_0 = 1
		[0 1]
	x_0 = 2
		[1 1]
x_1 = 2
	x_0 = 2
		[0]
	x_0 = 3
		[1]


# Hellinger Distance

http://www.pythonexample.com/code/hellinger-distance-decision-tree/

https://gist.github.com/larsmans/3116927

http://videolectures.net/ecmlpkdd08_cieslak_ldtf/?q=hellinger%20distance

Three ways of computing the Hellinger distance between two discrete
probability distributions using NumPy and SciPy.

In [14]:
import time

In [15]:
import numpy as np
from scipy.linalg import norm
from scipy.spatial.distance import euclidean
 
_SQRT2 = np.sqrt(2)     # sqrt(2) with default precision np.float64

def hellinger_dist(p, q):
    return np.sqrt(np.sum((np.sqrt(p) - np.sqrt(q)) ** 2)) / _SQRT2

In [16]:
def hellinger_dist(p, q):
    return np.sqrt(np.sum((np.sqrt(p) - np.sqrt(q)) ** 2)) / _SQRT2

In [17]:
# The input values
x1 = [0, 1, 1, 2, 2, 2, 1]
x2 = [0, 0, 1, 1, 1, 0, 1]

# The class
y = np.array([0, 0, 0, 1, 1, 0, 0])

In [18]:
# repeat = 1

# p = np.array([.05, .05, .1, .1, .2, .2, .3] * repeat)
# q = np.array([  0,   0,  0, .1, .3, .3 ,.3] * repeat)

# p /= p.sum()
# q /= q.sum()

# hellinger_dist(p=p, q=q)

In [19]:
# The input values
x1 = [0, 1, 1, 2, 2, 2, 1]
x2 = [0, 0, 1, 1, 1, 0, 1]

# The class
y = np.array([0, 0, 0, 1, 1, 0, 0])

In [20]:
def partition(a):
    return {c: (a==c).nonzero()[0] for c in np.unique(a)}

In [21]:
from pprint import pprint

def is_pure(s):
    return len(set(s)) == 1

def recursive_split(x, y):
    
    # If there could be no split, just return the original set
    if is_pure(y) or len(y) == 0:
        return y

    # We get attribute that gives the smallest distance
    distance = np.array([hellinger_dist(y/y.sum(), x_attr/x_attr.sum()) for x_attr in x.T])
    selected_attr = np.argmin(distance)

    # If the distance is very less, nothing has to be done, just return the original set
    if np.all(distance < 1e-6):
        return y

    # We split using the selected attribute
    sets = partition(x[:, selected_attr])

    res = {}
    for k, v in sets.items():
        y_subset = y.take(v, axis=0)
        x_subset = x.take(v, axis=0)

        res["x_%d = %d" % (selected_attr, k)] = recursive_split(x_subset, y_subset)

    return res

X = np.array([x1, x2]).T
hellinger = recursive_split(X, y)

In [22]:
def pretty_print(d, indent=0):
    for key, value in d.items():
        print('\t' * indent + str(key))
        if isinstance(value, dict):
            pretty(value, indent+1)
        else:
            print('\t' * (indent+1) + str(value))

In [23]:
pretty_print(hellinger)

x_1 = 0
	[0 0 0]
x_1 = 1
	x_0 = 1
		[0 0]
	x_0 = 2
		[1 1]
